In [3]:
import numpy as np


def newton_raphson_2_bus_PQ(P_spec, Q_spec, V, theta, Ybus, tol=1e-3, max_iter=20):
    """
    Newton-Raphson method for a 2-bus power system.

    Parameters:
        P_spec: Specified active power at Bus 2
        Q_spec: Specified reactive power at Bus 2
        V: Initial voltage magnitude array
        theta: Initial voltage angle array
        Ybus: Admittance matrix of the system
        tol: Tolerance for convergence
        max_iter: Maximum number of iterations

    Returns:
        V: Voltage magnitude array after convergence
        theta: Voltage angle array after convergence
    """

    def power_mismatch(V, theta, Ybus):
        P_calc = np.zeros(2)
        Q_calc = np.zeros(2)

        P_calc[1] = V[1] * V[0] * (Ybus[1, 0].real * np.cos(theta[1] - theta[0]) + Ybus[1, 0].imag * np.sin(theta[1] - theta[0])) + V[1] * V[1] * (Ybus[1, 1].real * np.cos(theta[1] - theta[1]) + Ybus[1, 1].imag * np.sin(theta[1] - theta[1]))
        Q_calc[1] = V[1] * V[0] * (Ybus[1, 0].real * np.sin(theta[1] - theta[0]) - Ybus[1, 0].imag * np.cos(theta[1] - theta[0])) + V[1] * V[1] * (Ybus[1, 1].real * np.sin(theta[1] - theta[1]) - Ybus[1, 1].imag * np.cos(theta[1] - theta[1]))

        #P_calc[1] = (V[1] * V[0] * (Ybus[1, 0].imag * np.sin(theta[1] - theta[0])))
        #Q_calc[1] = (V[1] * V[0] * (-Ybus[1, 0].imag * np.cos(theta[1] - theta[0]))) - V[1] * V[1] * Ybus[1, 1].imag

        P_mismatch = np.array([P_spec[1] - P_calc[1]])
        Q_mismatch = np.array([Q_spec[1] - Q_calc[1]])

        return np.concatenate((P_mismatch, Q_mismatch))

    def jacobian(V, theta, Ybus):
        J = np.zeros((2, 2))

        dP_dtheta = V[1] * V[0] * (-Ybus[1, 0].real * np.sin(theta[1] - theta[0]) + Ybus[1, 0].imag * np.cos(theta[1] - theta[0])) 
        dP_dV =  V[0] * (Ybus[1, 0].real * np.cos(theta[1] - theta[0]) + Ybus[1, 0].imag * np.sin(theta[1] - theta[0])) + 2* V[1] * (Ybus[1, 1].real * np.cos(theta[1] - theta[1]) + Ybus[1, 1].imag * np.sin(theta[1] - theta[1]))

        dQ_dtheta = V[1] * V[0] * (Ybus[1, 0].real * np.cos(theta[1] - theta[0]) + Ybus[1, 0].imag * np.sin(theta[1] - theta[0]))
        dQ_dV =  V[0] * (Ybus[1, 0].real * np.sin(theta[1] - theta[0]) - Ybus[1, 0].imag * np.cos(theta[1] - theta[0])) + 2* V[1] * (Ybus[1, 1].real * np.sin(theta[1] - theta[1]) - Ybus[1, 1].imag * np.cos(theta[1] - theta[1]))

        J[0, 0] = dP_dtheta
        J[0, 1] = dP_dV
        J[1, 0] = dQ_dtheta
        J[1, 1] = dQ_dV

        return J

    iter_count = 0
    while iter_count < max_iter:
        iter_count += 1

        mismatch = power_mismatch(V, theta, Ybus)
        if np.linalg.norm(mismatch, np.inf) < tol:
            print(f'Converged in {iter_count} iterations.')
            return V, theta

        J = jacobian(V, theta, Ybus)
        delta = np.linalg.solve(J, mismatch)

        theta[1] += delta[0]
        V[1] += delta[1]

    raise ValueError('Newton-Raphson did not converge')
def calcu( V, theta, Ybus):
    P1 = np.zeros(1)

    Q1 = np.zeros(1)


    P1[0] = V[0] * V[0] * (Ybus[0, 0].real * np.cos(theta[0] - theta[0]) + Ybus[0, 0].imag * np.sin(theta[0] - theta[0])) + V[0] * V[1] * (Ybus[0, 1].real * np.cos(theta[0] - theta[1]) + Ybus[0, 1].imag * np.sin(theta[0] - theta[1]))
    Q1[0] = V[0] * V[0] * (Ybus[0, 0].real * np.sin(theta[0] - theta[0]) - Ybus[0, 0].imag * np.cos(theta[0] - theta[0])) + V[0] * V[1] * (Ybus[0, 1].real * np.sin(theta[0] - theta[1]) - Ybus[0, 1].imag * np.cos(theta[0] - theta[1]))

    return P1, Q1
# Example usage:

# Specified power at Bus 2
P_spec = np.array([0, -2.8])
Q_spec = np.array([0, -0.6])

# Initial guess for voltage magnitudes and angles
V = np.array([1.0, 1.0])
theta = np.array([0.0, 0.0])

# Admittance matrix
Ybus = np.array([[10 - 20j,-10  + 20j],
                 [-10 + 20j,10  - 20j]])

# Run Newton-Raphson power flow
V, theta = newton_raphson_2_bus_PQ(P_spec, Q_spec, V, theta, Ybus)
P1, Q1 = calcu( V, theta, Ybus)
theta=theta/3.14*180
print("Voltage magnitudes: ", V)
print("Voltage angles: ", theta)
print("Active power: ", P1)
print("Reactive power: ", Q1)


Converged in 4 iterations.
Voltage magnitudes:  [1.         0.90553866]
Voltage angles:  [ 0.         -6.34340233]
Active power:  [2.99999713]
Reactive power:  [0.99999759]
